# Install  packages - No need
Use the condat_python3 kernel, not the pyspark one

In [1]:
import gc
import numpy as np
import pandas as pd
import warnings
import time
import gc
import os

instance_type = 'c5d4xlarge' # change this
results_bucket = f"s3://xdss-benchmarks/benchmarks" # change this

name = 'spark'
data_path = 'datasets/taxi_parquet'
output_file = f'{name}_{instance_type}.csv'
results_path = f"results/{output_file}"
results_bucket = f"{results_bucket}/{output_file}" 
benchmarks = {
    'run':[],
    'duration': [],
    'task': []   
}

long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90


def get_results(benchmarks=benchmarks):
    return pd.DataFrame.from_dict(benchmarks, orient='index').T

def persist():
    gc.collect()
    get_results(benchmarks).to_csv(results_path)
    os.system(f"aws s3 cp {results_path} {results_bucket}")
    
def benchmark(f, df, name, **kwargs):    
    for i in range(2):
        start_time = time.time()
        ret = f(df, **kwargs)
        benchmarks['duration'].append(time.time()-start_time)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]
          
def add_nan(name):
    for i in range(2):
        benchmarks['duration'].append(np.nan)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]

          


!mkdir -p results
!mkdir -p datasets
print(f"We test every benchmark twice and save both results")

We test every benchmark twice and save both results


# Benchmark

In [2]:
from pyspark import sql, SparkConf, SparkContext
import pyspark.sql.functions as f

conf = SparkConf().setAppName('Benchmarks')

# make sure you have enough memory for this
conf.setExecutorEnv('spark.executor.memory', '2g') 
conf.setExecutorEnv('spark.driver.memory', '12g')
sc = SparkContext(conf = conf)
sqlContext = sql.SQLContext(sc)
data = sqlContext.read.parquet(data_path)
print(f"size: {data.count()} with {len(data.columns)} columns")

size: 1173057928 with 18 columns


In [3]:
# read/ open
def open_file(df=None):
    return sqlContext.read.parquet(data_path)

benchmark(open_file, df=data, name='read_file')

read_file took: 0.0694112777709961


0.0694112777709961

In [4]:
def count(df=None):
    return df.count()

benchmark(count, df=data, name='count')

count took: 0.3143007755279541


0.3143007755279541

In [5]:
def mean(df):
    return df.select(f.mean('fare_amount')).collect()

benchmark(mean, df=data, name='mean')

mean took: 1.2139227390289307


1.2139227390289307

In [6]:
def standard_deviation(df):
    return df.select(f.stddev('fare_amount')).collect()

benchmark(standard_deviation, df=data, name='standard deviation')

standard deviation took: 1.8699445724487305


1.8699445724487305

Unlike other techonlogies, vaex can return columns, or subset of values to explore lazely, 
but becouse many of the other tecnologies crashed at this point, we return a scalar instead.

In [7]:
def mean_of_sum(df):
    return df.select(f.mean(df['fare_amount'] + df['trip_distance'])).collect()

benchmark(mean_of_sum, df=data, name='sum columns mean')
add_nan('sum columns')

sum columns mean took: 2.1918411254882812
sum columns took: nan


nan

In [8]:
def mean_of_product(df):
    return df.select(f.mean(df['fare_amount'] * df['trip_distance'])).collect()

benchmark(mean_of_product, df=data, name='product columns mean')
add_nan('product columns')

product columns mean took: 2.174720048904419
product columns took: nan


nan

In [9]:
def value_counts(df):
    return df.select('passenger_count').distinct().collect()

benchmark(value_counts, df=data, name='value counts')

value counts took: 4.221186399459839


4.221186399459839

In [10]:
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df['pickup_longitude']
    phi_1 = df['pickup_latitude']
    theta_2 = df['dropoff_longitude']
    phi_2 = df['dropoff_latitude']
    temp = (f.cos(theta_1)*np.pi/180) * (f.cos(theta_2)*np.pi/180) * (f.sin(phi_2-phi_1)/2*np.pi/180)**2
    expression = 2 * f.atan2(f.sqrt(temp), f.sqrt(1-temp))
    return df.select(f.mean(expression)).collect()

benchmark(mean_of_complicated_arithmetic_operation, df=data, name='arithmetic operation mean')
add_nan('arithmetic operation')

arithmetic operation mean took: 45.82709217071533
arithmetic operation took: nan


nan

In [11]:
def groupby_statistics(df):
    ret = df.groupby('passenger_count').agg(
        f.mean('fare_amount'),
        f.stddev('fare_amount'),
        f.mean('tip_amount'),
        f.stddev('tip_amount')
    )
    ret.take(3)
    return ret

benchmark(groupby_statistics, df=data, name='groupby statistics')

groupby statistics took: 16.114511728286743


16.114511728286743

In [12]:
other = groupby_statistics(data)

In [13]:
def join(df, other):
    ret = df.join(other, on = 'passenger_count')
    ret.take(3)
    return ret
    
# benchmark(join, data, name='join', other=other)
add_nan('join')

join took: nan


nan

In [14]:
def join_count(df, other):
    return df.join(other, on = 'passenger_count').count()

benchmark(join_count, data, name='join count', other=other)

join count took: 266.19249176979065


266.19249176979065

## Filtered data

Dask is not build to run on filter data like you would normally, so we will apply the same strategy

In [15]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

Prepare filtered data and deleted 0 MB


In [16]:
def filter_data(df):
    expr_filter = (df.pickup_longitude > long_min)  & (df.pickup_longitude < long_max) & \
              (df.pickup_latitude > lat_min)    & (df.pickup_latitude < lat_max) & \
              (df.dropoff_longitude > long_min) & (df.dropoff_longitude < long_max) & \
              (df.dropoff_latitude > lat_min)   & (df.dropoff_latitude < lat_max)
    ret = df[expr_filter]
    ret.take(3) # evaluate the filter
    return ret
benchmark(filter_data, data, name='filter data')

filter data took: 0.12418651580810547


0.12418651580810547

In [17]:
filtered = filter_data(data)

del data
print(f"cleaned {gc.collect()} mb")

cleaned 194 mb


In [18]:
benchmark(filter_data, filtered, name='filtered count')
benchmark(mean, filtered, name='filtered mean')
benchmark(standard_deviation, filtered, name='filtered standard deviation')
benchmark(mean_of_sum, filtered, name ='filtered sum columns mean')
add_nan('filtered sum columns')
benchmark(mean_of_product, filtered, name ='filtered product columns mean')
add_nan('filtered product columns')
benchmark(mean_of_complicated_arithmetic_operation, filtered, name='filtered arithmetic operation mean')
add_nan('filtered arithmetic operation')
benchmark(value_counts, filtered, name ='filtered value counts')
benchmark(groupby_statistics, filtered, name='filtered groupby statistics')
other = groupby_statistics(filtered)
add_nan('filtered join')
benchmark(join_count, filtered, name='filtered join count', other=other)
print(name)
get_results(benchmarks)

filtered count took: 0.09793996810913086
filtered mean took: 10.679100275039673
filtered standard deviation took: 11.177909851074219
filtered sum columns mean took: 13.869092226028442
filtered sum columns took: nan
filtered product columns mean took: 13.668078660964966
filtered product columns took: nan
filtered arithmetic operation mean took: 45.85776615142822
filtered arithmetic operation took: nan
filtered value counts took: 13.982243061065674
filtered groupby statistics took: 26.2817702293396
filtered join took: nan
filtered join count took: 276.03814697265625
spark


,run,duration,task
0,1,0.0742757,read_file
1,2,0.0694113,read_file
2,1,0.359038,count
3,2,0.314301,count
4,1,1.84555,mean
5,2,1.21392,mean
6,1,2.68757,standard deviation
7,2,1.86994,standard deviation
8,1,2.63623,sum columns mean
9,2,2.19184,sum columns mean
